In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
from torch import cuda

In [6]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [7]:
device

'cpu'

In [8]:
def QARetrieval(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data[output_col_name]
        self.text